In [ ]:
from PIL import Image, ImageEnhance
import os
import numpy as np
import cv2

# Paths
os.makedirs('./asl_data_res_aug')
input_folder = './_asl_dataset'  # Original dataset
output_folder = './asl_data_res_aug'  # Augmented dataset

# Ensure the output directory exists
os.makedirs(output_folder, exist_ok=True)

# Define augmentations
def augment_image(image):
    augmented_images = []

    # Convert to OpenCV format
    image_cv = np.array(image)
    
    # Rotation (-30 to +30 degrees)
    angles = [-30, -15, 15, 30]
    for angle in angles:
        augmented_images.append(image.rotate(angle))

    # Horizontal Flip
    augmented_images.append(image.transpose(Image.FLIP_LEFT_RIGHT))

    # Color jitter (brightness, contrast, saturation)
    enhancer = ImageEnhance.Color(image)
    augmented_images.append(enhancer.enhance(1.5))

    # **Noise (Gaussian Noise)**
    noise = np.random.normal(0, 25, image_cv.shape).astype(np.uint8)
    noisy_img = cv2.add(image_cv, noise)
    augmented_images.append(Image.fromarray(np.clip(noisy_img, 0, 255).astype(np.uint8)))

    # **Blur (Gaussian Blur)**
    blurred_img = cv2.GaussianBlur(image_cv, (5, 5), 0)
    augmented_images.append(Image.fromarray(blurred_img))

    # **Distortion (Perspective Warp)**
    rows, cols, _ = image_cv.shape
    src_pts = np.float32([[0, 0], [cols, 0], [0, rows], [cols, rows]])
    dst_pts = np.float32([[0, 0], [cols-10, 10], [10, rows-10], [cols, rows]])
    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped_img = cv2.warpPerspective(image_cv, matrix, (cols, rows))
    augmented_images.append(Image.fromarray(warped_img))

    return augmented_images

# Process all images and retain class structure
for dirpath, dirnames, filenames in os.walk(input_folder):
    for filename in filenames:
        class_label = os.path.basename(dirpath)  # Extract class name
        class_output_folder = os.path.join(output_folder, class_label)

        # Ensure class subdirectory exists
        os.makedirs(class_output_folder, exist_ok=True)

        image_path = os.path.join(dirpath, filename)

        # Open and resize image
        with Image.open(image_path) as img:
            img = img.resize((200, 200), Image.Resampling.LANCZOS)

            # Save original resized image in the correct class folder
            img.save(os.path.join(class_output_folder, filename))  

            # Apply augmentations and save in the same class folder
            augmented_images = augment_image(img)
            for i, aug_img in enumerate(augmented_images):
                aug_img.save(os.path.join(class_output_folder, f"aug_{i}_{filename}"))

print("Augmentation completed, and class structure retained!")  
